# **Simulación de sistema con escalamiento horizontal y ataques DDoS**

---

## Introducción

El objetivo principal de este informe es modelar y simular el comportamiento de un sistema web que implementa un esquema de balanceo de carga entre múltiples servidores, y que posee un sistema de escalamiento dinámico que le permite prender y apagar instancias del servidor de acuerdo a la medición de la cantidad de tráfico que está recibiendo. Dados distintos experimentos nos gustaría generar cierta intuición acerca de las problemáticas que se deben tener en cuenta a la hora de diseñar un sistema de estas características. ¿Qué tan potentes deben ser los servidores? ¿Me conviene tener pocos servidores con mucha capacidad o muchos servidores menos eficientes? ¿En qué momento decido prender/apagar un servidor dadas las mediciones de tráfico actuales?

### Escalamiento vertical y horizontal

El problema de la escalabilidad de un sistema informático refiere a la capacidad que tiene el sistema de aumentar la cantidad de recursos disponibles para poder lidiar con una mayor carga de trabajo. Hoy en día una persona que decide hacer a su sistema (en este trabajo el sistema es un servidor) escalable tiene dos opciones: escalar verticalmente, es decir agregar recursos a su servidor (CPU, ram, disco más rápido, etc.) o escalar horizontalmente, esto significa tener varias réplicas del servidor y distribuir los trabajos entrantes entre todos los servidores disponibles.

El escalado vertical es más sencillo de implementar ya que solamente requiere la instalación de los nuevos recursos sobre el sistema existente. Sin embargo tiene varias desventajas, la principal es que existe un límite físico de la cantidad de recursos que puede tener una sola computadora, y el sistema solo puede ser tan rápido como el mejor hardware/software disponible a la fecha.

El escalado horizontal no presenta estos problemas, en teoría siempre se pueden seguir creando réplicas del servidor, por lo que no hay límite (nuevamente teórico) para la capacidad de procesamiento de un sistema con estas características. Sin embargo adaptar un sistema para que pueda implementar un esquema de distribución de carga puede requerir un rediseño, ya que de entrada puede no ser apto para funcionar de manera distribuida. Por otro lado un sistema de este tipo es mucho más complejo, ya que se requieren de nuevos actores, como un balanceador de carga que tome la decisión de a qué réplica enviar el siguiente trabajo, se puede incluir un nuevo sistema que se encargue de monitorear la carga del sistema y decida dinámicamente si encender o apagar una nueva réplica.

En este trabajo nos enfocaremos mayormente en simular un sistema que balancea carga entre varios servidores idénticos


![full-web-stack](img/full-web-architecture.png)

No nos vamos a detener mucho en esta imagen, es más que nada para mostrar la complejidad que tiene hoy en día una aplicación web pensada para escalar horizontalmente (nosotros vamos a ver solamente los elementos 2 y 3 de este diagrama)

### Ataques (D)DoS



---

# **Modelado**

Nuestro modelo de servidor se comportará de la siguiente manera:

1. Tendremos dos generadores de tráfico, uno de tráfico normal y otro que simula un ataque DoS, estos generaran pulsos que llegan a la cola del sistema
2. La cola del sistema recibe estos pedidos, les asigna un identificador y los encola. Cuando a la cola le llegue un pedido de emisión de un pedido desencolará al más antiguo y lo enviará por un puerto de salida. Además la cola informará periódicamente su porcentaje de llenado y en caso de estar al 100% y recibir un paquete, lo descartará e informará el identificador que le corresponde.
3. El dispatcher es el encargado de distribuir los paquetes de la cola entre los servidores disponibles. Mantiene una cuenta de que servidores están prendidos y va pidiendo a la cola nuevos trabajos y repartiendolos entre los servidores que tiene disponibles.
4. El autoscaler monitorea la carga de la cola y decide en base a las mediciones si debe prender o apagar un servidor, además de avisar al dispatcher de los cambios de estado de dichos servidores.
5. Un servidor recibe señales del autoscaler y trabajos del dispatcher, procesándolos por una cantidad de tiempo aleatoria y emitiendo el identificador del trabajo una vez que fue procesado. 


A continuación daremos en mayor detalle la implementación de cada uno de estos agentes.

# Server


![server](img/server.png)

### Parámetros

* **initialStatus**: estado del servidor al inicio de la simulación (on|off)
* **distribution**: distribución que determina el tiempo de procesamiento de un trabajo (también acepta los parámetros que define la distribución elegida)
* **setupTime**: tiempo que tarda en estar operativo un servidor desde que le llega la señal de encendido

### Puertos

#### Entrada

* **job**: recibe el identificador del trabajo a procesar
* **powerSignal**: para indicar al servidor si debe prenderse/apagarse

#### Salida

* **done**: envía el id del trabajo procesado
* **ready**: avisa que ya está operacional/ que ya se apagó


### **Encendiendo el server**

#### **Transición externa**

```
si está apagado:
    marcar que se está prendiendo
    holdIn(setupTime)
caso contrario:
    error
```

#### **Función de output**
```
output(powerSignal, SEÑAL_PRENDIDO)
```

#### **Transición interna**
```
setear estado de servidor libre
desmarcar que se está prendiendo
passivate()
```

### **Apagando el server**


#### **Transición externa**
```
si no está procesando un trabajo:
    marcar servidor apagado
    holdIn(0)
si está procesando un trabajo:
    marcar trabajo final pendiente
    holdIn(tiempo de procesamiento restante)
si no:
    error
```

#### **Función de output**

```
si quedaba trabajo final pendiente:
    output(job, ID del paquete)
output(ready, SEÑAL_APAGADO)

```

#### **Transición interna**
```
si quedaba trabajo final pendiente:
    marcar servidor apagado
    desmarcar trabajo final pendiente
passivate()
```

### **Procesando un mensaje**

#### **Transición externa**

```
si está libre:
    tiempo_procesamiento := obtenerValor(distribucion)
    marcar servidor como ocupado
    holdIn(tiempo_procesamiento)
si no:
    error
```

#### **Función de output**
```
output(job, JOB_ID) 
```

#### **Transición interna**
```
marcar servidor libre
passivate()
```

## Queue

![queue](img/queue.png)

### Parámetros

* **size**: tamaño de la cola
* **currentSizeFrequency**: frecuencia con la cual informa su tamaño

### Puertos

#### Entrada

* **in**: recibe los nuevos elementos
* **emit**: indica que se debe enviar el siguiente elemento

#### Salida

* **out**: envía el id del elemento
* **discarded**: envía el id del elemento descartado por estar llena
* **queueLoad**: envía la carga de la cola*
* **loadAvg**: envía la carga de la cola (para experimentos)*

\* *queueLoad* envía la carga de la cola con la frecuencia indicada por el parámetro *currentSizeFrequency*, *loadAvg* marca el cambio instantáneo de la carga para poder después hacer la comparación.


### Transición externa
```
si llegó un nuevo paquete:
    si la cola está llena:
        marcar el paquete para descarte
        guardar tiempo restante hasta proximo envio de carga
        holdIn(0)
    si no:
        agregar paquete
        si tenía un envío pendiente:
            guardar tiempo restante hasta proximo envio de carga
            holdIn(0)
si llegó un pedido de emisión:
    si la cola está vacía:
        marcar envío pendiente
    si no:
        guardar tiempo restante hasta proximo envio de carga
        holdIn(0)
```
### Función de output
```
si hay que enviar la carga:
    output(queueLoad, #paquetes/tamaño)
si hay paquetes marcados para descarte:
    para cada paquete marcado:
        output(discarded, ID_paquete)
si hay que enviar paquete y la cola no está vacia:
    output(out, ID_paquete)
```
### Transición interna
```
si se mandó la carga de la cola:
    holdIn(currentSizeFrequency)
si no:
    reestablecer tiempo restante hasta próximo envío de carga

si envió un paquete:
    desencolar paquete
si se descartaron paquetes:
    eliminar paquetes descartados
```
---

# Scaler

![autoscaler](img/autoscaler.png)


## Parámetros

* **numberOfServers**: cantidad de servidores disponibles (prendidos y apagados)
* **loadLowerBound**: umbral de carga por debajo del cual se apaga un servidor
* **loadUpperBound**: umbral de carga por encima del cual se prende un servidor
* **loadUpdatesToBreakIdle**: cantidad de actualizaciones de la carga que espero después de prender/apagar un servidor hasta tomar otra decisión
* **exponentialWeight**: constante utilizada en el cálculo del promedio con decaimiento exponencial
* **server_i**: estado inicial del i-ésimo servidor

## Puertos

### Entrada

* **queueLoad**: recibe el valor de carga de la cola
* **serverResponse**: recibe la actualización del server (si ya se prendió/apagó)

### Salida

* **servers_i**: conexión con el i-ésimo servidor
* **serverStatus**: para enviar actualizaciones en el estado de los servidores

## Transición Externa
```
si llegó actualización del factor de carga:
    actualizarPromedioPonderado(nuevoValor)
    si ya se puede realizar una accion:
        si hace falta apagar un servidor:
            holdIn(0)
        si hace falta prender un servidor:
            holdIn(0)
        si no:
            passivate()
    si no:
        decrementar actualizaciones pendientes hasta proxima accion
si llegó información de un servidor:
    holdIn(0)
```

## Función de Output
```
si llegó información de un servidor que se prendio:
    output(serverStatus, SEÑAL_PRENDIDO)
    
si hace falta apagar un servidor:
    server = obtenerServidorPrendido()
    output(server, SEÑAL_APAGADO)
    output(serverStatus, SEÑAL_APAGADO)
si hace falta prender un servidor:
    server = obtenerServidorApagado()
    output(server, SEÑAL_PRENDIDO)
```

Lo importante a marcar de la función de output es la asimetría en el tiempo que se informa el estado de un servidor recientemente cambiado. Cuando se envía a una señal de encendido a un servidor es necesario esperar a tener confirmación de que se efectivamente se encendió hasta permitir que se le envíen paquetes. Por otro lado no queremos que se envie un paquete a un servidor que comenzó a apagarse, por lo que no esperamos confirmación y avisamos inmediatamente del cambio del estado.

## Transición Interna
```
si llegó actualización de servidor:
    actualizar registro de estado del servidor
    desmarcar actualización de servidor
si hace falta prender o apagar un servidor:
    resetear la cantidad de actualizaciones pendientes hasta próxima acción
    
passivate()
```


---
# Dispatcher

![dispatcher](img/dispatcher.png)


## Parámetros
* **numberOfServers**: cantidad de servidores disponibles (prendidos y apagados)
* **server_i**: estado inicial del i-ésimo servidor

## Puertos

### Entrada

* **newJob**: recibe id del nuevo paquete que llega de la cola
* **jobDone**: recibe el id del paquete procesado por un servidor
* **serverStatus**: recibe actualizaciones del estado de los servidores

### Salida

* **requestJob**: para pedir un nuevo paquete
* **server_i**: para enviar los paquetes al i-ésimo servidor

## Transición Externa
```
si llegó un nuevo trabajo:
    si hay un servidor libre:
        holdIn(0)
    si me quedan servidores libres:
        marcar para pedir trabajo nuevo
si se completó un trabajo:
    si el servidor que envió la respuesta estaba marcado para apagarse:
        marcar servidor como apagado
    si no:
        marcar servidor como libre
        marcar para pedir trabajo nuevo
si llegó la actualización del estado de un servidor:
    si llegó actualización prendido y tengo el servidor apagado:
        marcar servidor como prendido
        marcar para pedir trabajo nuevo
    si llegó actualización apagado y tengo el servidor prendido:
        si está procesando un trabajo:
            marco para apagarlo cuando termine
        si no:
            marco servidor como apagado
    si no:
        error
```

## Función de Output
```
si llegó un trabajo:
    server := serverLibre()
    output(server, ID_paquete)
si tengo que pedir trabajo:
    output(requestJob, SEÑAL)
```

## Transición Interna
```
si llegó un trabajo:
    servidor := serverLibre()
    marco servidor como ocupado

si tenía marcado para pedir trabajo nuevo:
    desmarco para pedir trabajo nuevo
    
passivate()
```

---
# Attacker / (D)DoS Traffick Generator

![attacker](../report/img/attacker.png)

## Puertos

### Salida

* **attack**: envía el siguiente paquete

## Transición Externa
```
    pass
```
## Función de Output
```
    output(attack, ID_paquete)
```
## Transición Interna
```
    siguiente_envio := obtenerDeltaSiguienteEnvio()
    holdIn(siguiente_envio)
```

---



---
## Visualización de modelo completo

![full_model](img/DDoS-Sim-Diagram.png)